In [66]:
from http import HTTPStatus
from datetime import datetime
import json
from traceback import TracebackException

class WidgetException(Exception):
    """Base exception for the Widget application."""
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'A Widget exception occurred.'
    user_err_msg = 'An unexpected error occurred.'
    
    def __init__(self, *args, user_err_msg=None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
            
    @property
    def traceback(self):
        return TracebackException.from_exception(self).format()
            
    def log_exception(self):
        print(f'{self.__class__.__name__} occurred at {datetime.utcnow().isoformat()}')
        print(f'Internal error message: {self.internal_err_msg}')
        print(f'User error message: {self.user_err_msg}')
        print(f'HTTP status code: {self.http_status}, {self.http_status.name}')
        for line in self.traceback:
            print(line)
        
    def to_json(self):
        return json.dumps({
            'exception_type': self.__class__.__name__,
            'error_message': self.user_err_msg,
            'http_status_name': self.http_status.name,
            'http_status_code': self.http_status.value
        })
    
class SupplierException(WidgetException):
    """Exceptions caused by issues with widget supply."""
    
    internal_err_msg = 'A supplier exception occurred.'
    user_err_msg = 'An unexpected supplier error occurred.'    
    
    
class EOLException(SupplierException):
    """Product is EOL (end of life) and no longer manufactured."""
    
    internal_err_msg = 'Product EOL.'
    user_err_msg = 'This product is EOL.'
    
class ProductionDelayException(SupplierException):
    """Product's production has been delayed."""
    
    internal_err_msg = 'Production delayed.'
    user_err_msg = 'This item is experiencing a production delay.'
    
class ShippingDelayException(SupplierException):
    """Product is delayed in shipping."""
    
    internal_err_msg = 'Shipping delay.'
    user_err_msg = 'This item is delayed in shipping.'
    
class CheckoutException(WidgetException):
    """Exceptions caused during the checkout process."""
    
    internal_err_msg = 'Checkout exception.'
    user_err_msg = 'An unexpected error occurred during checkout.'
    
class InventoryException(CheckoutException):
    """Exceptions caused by issues with inventory."""
    
    internal_err_msg = 'Inventory exception.'
    user_err_msg = 'There is an issue with the inventory of this item.'
    
class StockOutException(InventoryException):
    """Product is out of stock."""
    
    internal_err_msg = 'Product out of stock.'
    user_err_msg = 'This product is out of stock.'
    
class PricingException(CheckoutException):
    """Checkout issues related to pricing."""
    
    internal_err_msg = 'Pricing issue.'
    user_err_msg = 'There is an issue with the pricing of this item.'
    
    http_status = HTTPStatus.BAD_REQUEST
    
class InvalidCouponCodeException(PricingException):
    """Supplied coupon code is invalid."""
    
    internal_err_msg = 'Invalid coupon code.'
    user_err_msg = 'The supplied coupon code is invalid.'
    
class CouponStackingException(PricingException):
    """Exception when trying to stack multiple coupons."""
    
    internal_err_msg = 'Cannot stack multiple coupons.'
    user_err_msg = 'Stacking multiple coupons is not allowed.'

In [54]:
WidgetException().internal_err_msg

'A Widget exception occurred.'

In [43]:
ex = CouponStackingException('I have no idea what is happening here.', user_err_msg='Everything is under control!')

In [48]:
tb = TracebackException.from_exception(ex)

In [52]:
list(tb.format())

['CouponStackingException: I have no idea what is happening here.\n']

In [70]:
try:
    raise StockOutException
except:
    try:
        raise CouponStackingException('I have no idea what is happening here.', user_err_msg='Everything is under control!')
    except CouponStackingException as ex:
        ex.log_exception()
    
                

CouponStackingException occurred at 2021-04-10T03:52:43.514114
Internal error message: I have no idea what is happening here.
User error message: Everything is under control!
HTTP status code: 400, BAD_REQUEST
Traceback (most recent call last):

  File "<ipython-input-70-b8418681f410>", line 2, in <module>
    raise StockOutException

StockOutException: Product out of stock.


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "<ipython-input-70-b8418681f410>", line 5, in <module>
    raise CouponStackingException('I have no idea what is happening here.', user_err_msg='Everything is under control!')

CouponStackingException: I have no idea what is happening here.



In [72]:
ex = CouponStackingException("Stuff's broke, mang.", 'two', 'three', user_err_msg='Bollocks!')

In [73]:
ex.user_err_msg

'Bollocks!'

In [74]:
ex.internal_err_msg

"Stuff's broke, mang."

In [77]:
for line in ex.traceback:
    print(line)

CouponStackingException: ("Stuff's broke, mang.", 'two', 'three')



In [64]:
PricingException().user_err_msg

'There is an issue with the pricing of this item.'

In [65]:
json.loads(ex.to_json())

{'exception_type': 'CouponStackingException',
 'error_message': 'Bollocks!',
 'http_status_name': 'BAD_REQUEST',
 'http_status_code': 400}

In [38]:
datetime.utcnow().isoformat()

'2021-04-10T02:59:55.624866'

In [3]:
WidgetException().http_status

<HTTPStatus.INTERNAL_SERVER_ERROR: 500>

In [16]:
CouponStackingException.http_status

<HTTPStatus.BAD_REQUEST: 400>